# OA Analysis
*This file is to be used to perform analysis on member's data to evaluate the current status of the data and to determine what cleanup must be performed before OA can be implemented.*

## Download data
*The first step to complete is to download the customer data from their server.  These files should reside in your **C:\Users\  [username]\Downloads** directory.*

## Enter Customer Name and Number
*For this script to execute properly you must enter the customer's name and member number.*

In [1]:
name = input("Enter the member name: ")
number = input("Enter the member number: ")

Enter the member name: Kandiyohi Power Cooperative
Enter the member number: 25058


## Create the directory in OneDrive
*This step creates a folder in OneDrive in the format of "[number] [name]"*

In [35]:
import os
from datetime import date
import calendar
month = calendar.month_abbr[date.today().month]
year = date.today().year
day = date.today().day
username = os.environ["USERNAME"]
onedrivedir = "C:/Users/"+ username +"/OneDrive - National Information Solutions Cooperative/"
downloadsdir = "C:/Users/"+ username +"/Downloads/"
projectsdir = "C:/Projects/"
backupsdir = "C:/Database Backups/"
ivuemappingdir = "C:/ivue_mapping_staking/"
coopsdir = r"\\mofs\sdesvr\Support\MDM-DA\Coops"
companydir = str(number) + " " + name + "/"
monthdir = month + " " + str(year) + "/"


if not os.path.exists(onedrivedir + companydir):
    os.mkdir(onedrivedir + companydir)

if not os.path.exists(onedrivedir + companydir + monthdir):
    os.mkdir(onedrivedir + companydir + monthdir)
    
if not os.path.exists(projectsdir + companydir + monthdir):
    os.mkdir(projectsdir + companydir + monthdir)

## Move the files
*This portion moves the backup file to your OneDrive account and unzips it.*

In [3]:
def movefile(filefrom, fileto):    
    import gzip, shutil

    if os.path.exists(filefrom):
        if os.path.exists(fileto):
            os.remove(fileto)
            os.rename(filefrom, fileto)
        else:
            os.rename(filefrom, fileto)
    else:
        print("File " + filefrom +" not found.")
    return

*Move the BAK.GZ database file to your OneDrive folder.*

In [4]:
gzfilename = "gs"+ str(number) +"_snapshot.bak.gz"

gzbakfrom = downloadsdir + gzfilename
gzbakto = onedrivedir + companydir + monthdir + gzfilename

movefile(gzbakfrom,gzbakto)

*Unzips the BAK.GZ database file to the **C:\Database Backups\** directory.*

In [47]:
import gzip,shutil

filename, file_extension = os.path.splitext(gzbakto)
if not os.path.exists(projectsdir + companydir + monthdir):
    os.mkdir(projectsdir + companydir + monthdir)
    
if file_extension == ".gz":
    with gzip.open(gzbakto, 'r') as f_in, open(projectsdir + companydir + monthdir + gzfilename[:-3], 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)

*Moves the windmil ZIP file to the **OneDrive** directory.*

In [6]:
filename = "gs"+ str(number) +"_windmil_export.zip"

filefrom = downloadsdir + filename
fileto = onedrivedir + companydir + monthdir + filename

movefile(filefrom,fileto)

*Moves the windmil properties file to the **OneDrive** directory.*

In [7]:
filename = "windmilExport.properties"

filefrom = downloadsdir + filename
fileto = onedrivedir + companydir + monthdir + filename

movefile(filefrom,fileto)

*Moves the VUData file to the **OneDrive** directory.*

In [8]:
filename = "VUData.mdb"

filefrom = downloadsdir + filename
fileto = onedrivedir + companydir + monthdir + filename

movefile(filefrom,fileto)

*Moves the MXD file to the **OneDrive** directory.*

In [9]:
filename = "gs"+ str(number) +".mxd"

filefrom = downloadsdir + filename
fileto = onedrivedir + companydir + monthdir + filename

movefile(filefrom,fileto)

*Moves the MeterExport CSV file to the **OneDrive** directory and adds the header, saves as a XLSX.*

In [10]:
import pandas as pd

if os.path.exists(downloadsdir + "MeterExport.csv"):
    colnames = ["Mtr Num","Billing Acct Num","Phase","Map Loc","Address","Address","City","State","Zip",""," ","Med alert code","Outage priority","Gen Code","Serv Status","Connect date","Disconnect date","Transf Number","Service Map Loc","Local Name","Xfmr_622-03-01"
]

    meterexport = pd.read_csv(downloadsdir + "MeterExport.csv", names=colnames).to_excel(onedrivedir + companydir + monthdir + "MeterExport.xlsx", index=False)
    os.remove(downloadsdir + "MeterExport.csv")
else:
    print("File " + downloadsdir + "MeterExport.csv not found.")

C:\Users\aakins\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


*Moves the TransformerExport CSV file to the **OneDrive** directory, saves as a XLSX.*

In [25]:
if os.path.exists(downloadsdir + "Transformer_Export.csv"):
    transformerexport = pd.read_csv(downloadsdir + "Transformer_Export.csv").to_excel(onedrivedir + companydir + monthdir + "Transformer_Export.xlsx", index=False)
    os.remove(downloadsdir + "Transformer_Export.csv")
else:
    print("File " + downloadsdir + "Transformer_Export.csv not found.")

*Copy the OAMapWiseDataReview template file to the **OneDrive** directory, saves as a XLSX.*

In [12]:
import openpyxl

xltemplatefile = coopsdir + "\OAMapWiseDataReview_Master_temp.xlsx"
xlanalysisfile = onedrivedir + companydir + monthdir + "gs" + number + "-OAMapWiseDataReview-" + str(day) + str(month) + str(year) + ".xlsx"

if os.path.exists(xltemplatefile):
    mywb = openpyxl.load_workbook(xltemplatefile)
    mywb.save(xlanalysisfile)
else:
    print("File " + xltemplatefile + " not found.")

## Restore the DB
*Restores the DB to the local SQL server.*

In [62]:
import pyodbc
import os
import time

conn = pyodbc.connect("Driver={SQL Server};Server=.\SQLEXPRESS;Database=master;Trusted_Connection=yes", autocommit = True)
conn.timeout = 60
cursor = conn.cursor()
file_list = []


def get_filelistonly(bak_file):
    sqlcommand = r"""
                    RESTORE filelistonly FROM DISK = N'{bak_file}'
                 """.format(bak_file=bak_file)
    cursor.execute(sqlcommand)
    rows = cursor.fetchall()

    for row in rows:
        fname = row[0]
        fext = os.path.splitext(row[1])[1]
        if "." not in fext:
            raise ValueError("No extension found in row")
        file_list.append({"fname": fname, "fext": fext})
    return file_list

def get_restore_command(new_db, bak_file, file_list):
    r = None
    if len(file_list) > 0:
        sqlcommand = r"""RESTORE DATABASE {new_db} FROM DISK = N'{bak_file}'
                        WITH
                        FILE = 1,
                     """.format(new_db=new_db, bak_file=bak_file)
        sqlcommand = sqlcommand + ", \n".join(("MOVE N'{fname}' TO N'{projectsdir}{number} {name}\{new_db}{fext}'".format(fname=fl['fname'], fext=fl['fext'], new_db=new_db, number=number, name=name, projectsdir=projectsdir) for fl in file_list))
        sqlcommand = sqlcommand + ", NOUNLOAD, REPLACE, STATS = 5"
        r = sqlcommand
        sqlcommand = sqlcommand.replace("/" , "\\")
        print(sqlcommand)
        try:
            cursor.execute(sqlcommand)
            while cursor.nextset():
                pass
        except:
            pass
    return r

rows_empty = ()
backup_file = projectsdir + companydir + monthdir + gzfilename[:-3]
file_list = get_filelistonly(backup_file)

r = get_restore_command("gs" + number, backup_file, file_list)
time.sleep(15)
conn.close()

RESTORE DATABASE gs25058 FROM DISK = N'C:\Projects\25058 Kandiyohi Power Cooperative\Apr 2019\gs25058_snapshot.bak'
                        WITH
                        FILE = 1,
                     MOVE N'gs25058' TO N'C:\Projects\25058 Kandiyohi Power Cooperative\gs25058.mdf', 
MOVE N'gs25058_log' TO N'C:\Projects\25058 Kandiyohi Power Cooperative\gs25058.ldf', NOUNLOAD, REPLACE, STATS = 5


## Begins the analysis
*Defines the function to count the total rows*

In [63]:
def sumrows(table):
    cursor.execute("SELECT COUNT(*) FROM {table}".format(table=table))
    row = cursor.fetchone()
    totalrows = str(row[0])
    return totalrows

*Defines the function to determine unique IDs.*

In [64]:
def uniqueid(idcolumn, table):
    c = "Error."
    facility_id = False
    for row in cursor.columns(table=table):
        if "gs_facility_id" in row:
            facility_id = True
    global cell_style 
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    rows = cursor.columns(table=table)
    if int(sumrows(table)) == 0:
                cell_style = 'Bad'
                c = "This table is empty."
    else:
        for row in cursor.columns(table=table):
            if idcolumn in row:
                r = r""
                sqlcommand = r.join(("SELECT {table}.OBJECTID, {table}.{idcolumn}".format(idcolumn=idcolumn,table=table)))
                if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                    sqlcommand = r.join((sqlcommand, """,{table}.gs_facility_id, Count({table}.OBJECTID) AS [Count] 
                        FROM {table} WHERE ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '') 
                        GROUP BY {table}.{idcolumn}, {table}.OBJECTID""".format(idcolumn=idcolumn,table=table)))
                else:
                    sqlcommand = r.join((sqlcommand, """, Count({table}.OBJECTID) AS [Count] 
                        FROM {table} WHERE ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '') 
                        GROUP BY {table}.{idcolumn}, {table}.OBJECTID""".format(idcolumn=idcolumn,table=table)))
                if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                    sqlcommand = r.join((sqlcommand, """,{table}.gs_facility_id 
                        ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC, {table}.OBJECTID
                        """.format(idcolumn=idcolumn,table=table)))
                else:
                    sqlcommand = r.join((sqlcommand, """ 
                        ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC, {table}.OBJECTID
                        """.format(idcolumn=idcolumn,table=table)))
                cursor.execute(sqlcommand)
                rows = cursor.fetchall()
                totalrows = sumrows(table)
                if not rows:
                    cell_style = 'Normal'
                    cell_alignment = 'False'
                    c = "All IDs are populated."
                else:
                    if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                        countid = str(sum(e[3] for e in rows))
                    else:
                        countid = str(sum(e[2] for e in rows))
                    percent = int(round(int(countid)/int(totalrows)*100))
                    cell_style = 'Bad'
                    c = str(percent) + "% (" + str(countid) + ")" + " blank values."
                    r = r""
                    sqlcommand = r.join(("SELECT {table}.OBJECTID, {table}.{idcolumn}".format(idcolumn=idcolumn,table=table)))
                    if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                        sqlcommand = r.join((sqlcommand, """,{table}.gs_facility_id, Count({table}.OBJECTID) AS [Count] 
                            FROM {table} WHERE ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '') 
                            GROUP BY {table}.{idcolumn}, {table}.OBJECTID""".format(idcolumn=idcolumn,table=table)))
                    else:
                        sqlcommand = r.join((sqlcommand, """, Count({table}.OBJECTID) AS [Count] 
                            FROM {table} WHERE ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '') 
                            GROUP BY {table}.{idcolumn}, {table}.OBJECTID""".format(idcolumn=idcolumn,table=table)))
                    if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                        sqlcommand = r.join((sqlcommand, """,{table}.gs_facility_id 
                            ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC, {table}.OBJECTID
                            """.format(idcolumn=idcolumn,table=table)))
                    else:
                        sqlcommand = r.join((sqlcommand, """ 
                            ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC, {table}.OBJECTID
                            """.format(idcolumn=idcolumn,table=table)))
                    df = pd.read_sql_query(sqlcommand, conn)
                    sheetname = category + "-Blanks"
                    createsheet(sheetname)
                    writedf(df, sheetname)
                break
            else:
                cell_style = 'Neutral'
                c = "This field needs to be added to the database."  
    return c

In [65]:
def duplicateid(idcolumn, idcolumn1, table):
    c = "Error."
    global cell_style
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    rows = cursor.columns(table=table)
    for row in cursor.columns(table=table):
        if "gs_facility_id" in row:
            facility_id = True
    if int(sumrows(table)) == 0:
        cell_style = 'Bad'
        c = "This table is empty."
    else:
        for row in cursor.columns(table=table):
            if idcolumn in row:
                for row in cursor.columns(table=table):
                    if idcolumn1 in row:
                        sqlcommand = r"""SELECT DISTINCT {table}.{idcolumn1}, {table}.{idcolumn}
                                        FROM {table}
                                        WHERE ((({table}.{idcolumn}) In (SELECT [{idcolumn}] FROM [{table}] As Tmp GROUP BY [{idcolumn}] HAVING Count(*)>1 )))
                                        ORDER BY {table}.{idcolumn};
                                    """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)
                        cursor.execute(sqlcommand)
                        row = cursor.fetchall()
                        if not row:
                            cell_style = 'Normal'
                            cell_alignment = 'False'
                            c = "No duplicates found."
                        else:
                            cell_style = 'Bad'
                            c = str(len(row)) + " duplicates exist."
                            sqlcommand = r"""SELECT DISTINCT {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}
                                    FROM {table}
                                    WHERE ((({table}.{idcolumn}) In (SELECT [{idcolumn}] FROM [{table}] As Tmp GROUP BY [{idcolumn}] HAVING Count(*)>1 )))
                                    GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}
                                    ORDER BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}
                                    """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)
                            r = r""
                            sqlcommand = r.join(("SELECT DISTINCT {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, """, {table}.gs_facility_id 
                                    FROM {table} 
                                    WHERE ((({table}.{idcolumn}) 
                                        In (SELECT [{idcolumn}] 
                                        FROM [{table}] As Tmp 
                                        GROUP BY [{idcolumn}] HAVING Count(*)>1 )))
                                    GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}, {table}.gs_facility_id 
                                    ORDER BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}, {table}.gs_facility_id """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            else:
                                sqlcommand = r.join((sqlcommand, """ FROM {table} 
                                    WHERE ((({table}.{idcolumn}) 
                                        In (SELECT [{idcolumn}] 
                                        FROM [{table}] As Tmp 
                                        GROUP BY [{idcolumn}] HAVING Count(*)>1 )))
                                    GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}
                                    ORDER BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}""".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            df = pd.read_sql_query(sqlcommand, conn)
                            sheetname = category + "-" + idcolumn[len("gs_"):]
                            createsheet(sheetname)
                            writedf(df, sheetname)
                        break
                    else:
                        cell_style = 'Neutral'
                        c = "The field ''" + str(idcolumn1) + "' needs to be added to the database."
            else:
                cell_style = 'Neutral'
                c = "The field ''" + str(idcolumn) + "' needs to be added to the database."
    return c

In [66]:
def duplicate_xfmr(idcolumn, idcolumn1, table):
    c = "Error."
    facility_id = False
    global cell_style
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    rows = cursor.columns(table=table)
    for row in cursor.columns(table=table):
        if "gs_facility_id" in row:
            facility_id = True
    if int(sumrows(table)) == 0:
        cell_style = 'Bad'
        c = "This table is empty."
    else:
        for row in cursor.columns(table=table):
            if idcolumn in row:
                for row in cursor.columns(table=table):
                    if idcolumn1 in row:
                        r = r""
                        sqlcommand = r.join(("SELECT {table}.OBJECTID, ".format(table=table)))
                        if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                            sqlcommand = r.join((sqlcommand, "{table}.gs_facility_id, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                        else:
                            sqlcommand = r.join((sqlcommand, "{table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                        if not ((idcolumn == "gs_phase") or (idcolumn1 == "gs_phase")):
                            sqlcommand = r.join((sqlcommand, ",{table}.gs_phase FROM {table} WHERE ((({table}.{idcolumn}) In (SELECT {idcolumn} FROM {table} As Tmp GROUP BY {idcolumn}, {idcolumn1}, gs_phase HAVING Count(*)>1 ))) GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn, idcolumn1=idcolumn1, table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                        else:
                            sqlcommand = r.join((sqlcommand, " FROM {table} WHERE ((({table}.{idcolumn}) In (SELECT {idcolumn} FROM {table} As Tmp GROUP BY {idcolumn}, {idcolumn1} HAVING Count(*)>1 ))) GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn, idcolumn1=idcolumn1, table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                        if not ((idcolumn == "gs_phase") or (idcolumn1 == "gs_phase")):
                            sqlcommand = r.join((sqlcommand, ", {table}.gs_phase ORDER BY {table}.{idcolumn}, {table}.OBJECTID, {table}.{idcolumn1}, {table}.gs_phase".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                        else:
                            sqlcommand = r.join((sqlcommand," ORDER BY {table}.{idcolumn}, {table}.OBJECTID, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                        cursor.execute(sqlcommand)
                        row = cursor.fetchall()
                        if not row:
                            cell_style = 'Normal'
                            cell_alignment = 'False'
                            c = "No duplicates found."
                        else:
                            cell_style = 'Bad'
                            c = str(len(row)) + " duplicates exist."
                            r = r""
                            sqlcommand = r.join(("SELECT {table}.OBJECTID, ".format(table=table)))
                            if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                sqlcommand = r.join((sqlcommand, "{table}.gs_facility_id, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            else:
                                sqlcommand = r.join((sqlcommand, "{table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                            if not ((idcolumn == "gs_phase") or (idcolumn1 == "gs_phase")):
                                sqlcommand = r.join((sqlcommand, ",{table}.gs_phase FROM {table} WHERE ((({table}.{idcolumn}) In (SELECT {idcolumn} FROM {table} As Tmp GROUP BY {idcolumn}, {idcolumn1}, gs_phase HAVING Count(*)>1 ))) GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn, idcolumn1=idcolumn1, table=table)))
                                if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                    sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                            else:
                                sqlcommand = r.join((sqlcommand, " FROM {table} WHERE ((({table}.{idcolumn}) In (SELECT {idcolumn} FROM {table} As Tmp GROUP BY {idcolumn}, {idcolumn1} HAVING Count(*)>1 ))) GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.{idcolumn1}".format(idcolumn=idcolumn, idcolumn1=idcolumn1, table=table)))
                                if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                    sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                            if not ((idcolumn == "gs_phase") or (idcolumn1 == "gs_phase")):
                                sqlcommand = r.join((sqlcommand, ", {table}.gs_phase ORDER BY {table}.{idcolumn}, {table}.OBJECTID, {table}.{idcolumn1}, {table}.gs_phase".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                                if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                    sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                            else:
                                sqlcommand = r.join((sqlcommand," ORDER BY {table}.{idcolumn}, {table}.OBJECTID, {table}.{idcolumn1}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                                if (facility_id == True and (not(idcolumn == "gs_facility_id" or idcolumn1 == "gs_facility_id"))):
                                    sqlcommand = r.join((sqlcommand, ", {table}.gs_facility_id".format(table=table)))
                            df = pd.read_sql_query(sqlcommand, conn)
                            sheetname = category + "-" + idcolumn[len("gs_"):]
                            createsheet(sheetname)
                            writedf(df, sheetname)
                        break
                    else:
                        cell_style = 'Neutral'
                        c = "The field ''" + str(idcolumn1) + "' needs to be added to the database."
            else:
                cell_style = 'Neutral'
                c = "The field ''" + str(idcolumn) + "' needs to be added to the database."
    return c

In [67]:
def fieldsummary(idcolumn, table):
    c = "Error."
    global cell_style 
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    facility_id = False
    for row in cursor.columns(table=table):
        if "gs_facility_id" in row:
            facility_id = True
    for row in cursor.columns(table=table):
        if idcolumn in row:
            file_list = []

            sqlcommand = r"""SELECT {table}.{idcolumn}, Count({table}.OBJECTID) AS [Count]
                        FROM {table}
                        GROUP BY {table}.{idcolumn}
                        ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC
                        """.format(idcolumn=idcolumn,table=table)
            cursor.execute(sqlcommand)
            row = cursor.fetchall()
            totalrows = sumrows(table)
            if not row:
                cell_style = 'Bad'
                c = "This table is empty."
            else:
                if int(sumrows(table)) == int(row[0][1]):
                    cell_style = 'Normal'
                    cell_alignment = 'False'
                    c = "All fields are populated with '" + str(row[0][0]) +"'."
                    if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                            cell_style = 'Bad'
                else:
                    cell_style = 'Normal'
                    for row in row:
                        if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                            cell_style = 'Bad'
                        fcol = row[0]
                        fnum = row[1]
                        fper = int(round(int(fnum)/int(totalrows)*100))
                        file_list.append({"fcol": fcol, "fnum": fnum, "fper": fper})
                    cell_alignment = 'True'
                    c = r""        
                    c = c.join(("Populated with {fper}% ({fnum}) '{fcol}'.\n".format(fcol=fl['fcol'], fnum=fl['fnum'], fper=fl['fper']) for fl in file_list))
                    if cell_style == 'Bad':

                        r = r""
                        sqlcommand = r.join(("SELECT {table}.OBJECTID, {table}.{idcolumn}".format(idcolumn=idcolumn,table=table)))
                        if (facility_id == True and (not(idcolumn == "gs_facility_id"))):
                            sqlcommand = r.join((sqlcommand, """,{table}.gs_facility_id 
                                FROM {table}
                                WHERE ({table}.{idcolumn} IS null 
                                    OR {table}.{idcolumn} = TRY_CONVERT(numeric, '') 
                                    OR {table}.{idcolumn} LIKE '%UNK%')
                                    GROUP BY {table}.OBJECTID, {table}.{idcolumn}, {table}.gs_facility_id
                                    ORDER BY {table}.OBJECTID, {table}.{idcolumn}, {table}.gs_facility_id""".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                        else:
                            sqlcommand = r.join((sqlcommand, """ FROM {table}
                                WHERE ({table}.{idcolumn} IS null 
                                    OR {table}.{idcolumn} = TRY_CONVERT(numeric, '') 
                                    OR {table}.{idcolumn} LIKE '%UNK%')
                                    GROUP BY {table}.OBJECTID, {table}.{idcolumn}
                                    ORDER BY {table}.OBJECTID, {table}.{idcolumn}""".format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)))
                        df = pd.read_sql_query(sqlcommand, conn)
                        sheetname = category + "-" + idcolumn[len("gs_"):]
                        createsheet(sheetname)
                        writedf(df, sheetname)
            break
        else:
            cell_style = 'Neutral'
            c = "This field needs to be added to the database."
    
    return c

In [68]:
def missingfield(idcolumn, table):
    c = "Error."
    global cell_style 
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    for row in cursor.columns(table=table):
        if idcolumn in row:
            file_list = []

            sqlcommand = r"""SELECT {table}.{idcolumn}, Count({table}.OBJECTID) AS [Count]
                        FROM {table}
                        WHERE ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '')
                        GROUP BY {table}.{idcolumn}
                        ORDER BY {table}.{idcolumn}, Count({table}.OBJECTID) DESC
                        """.format(idcolumn=idcolumn,table=table)
            cursor.execute(sqlcommand)
            row = cursor.fetchall()
            totalrows = sumrows(table)
            
            if int(sumrows(table)) == int(row[0][1]):
                cell_style = 'Normal'
                cell_alignment = 'False'
                c = "All fields are populated with '" + str(row[0][0]) +"'."
                if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                            cell_style = 'Bad'
            else:
                for row in row:
                    if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                        cell_style = 'Bad'
                    fcol = row[0]
                    fnum = row[1]
                    fper = int(round(int(fnum)/int(totalrows)*100))
                    file_list.append({"fcol": fcol, "fnum": fnum, "fper": fper})
                cell_alignment = 'True'
                c = r""        
                c = c.join(("Populated with {fper}% ({fnum}) '{fcol}'.\n".format(fcol=fl['fcol'], fnum=fl['fnum'], fper=fl['fper']) for fl in file_list))
            break
        else:
            cell_style = 'Neutral'
            c = "This field needs to be added to the database."
    
    return c

In [69]:
def neutral(idcolumn, table):
    c = "Error."
    global cell_style 
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    for row in cursor.columns(table=table):
        if idcolumn in row:
            file_list = []

            sqlcommand = r"""SELECT {table}.{idcolumn}, {table}.gs_subtype_cd, Count({table}.OBJECTID) AS [Count]
                        FROM {table}
                        WHERE {table}.gs_subtype_cd =  1
                        AND ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '' OR {table}.{idcolumn} LIKE '%UNK%')
                        GROUP BY {table}.{idcolumn}, {table}.gs_subtype_cd
                        ORDER BY {table}.{idcolumn}, {table}.gs_subtype_cd, Count({table}.OBJECTID) DESC
                        """.format(idcolumn=idcolumn,table=table)
            cursor.execute(sqlcommand)
            row = cursor.fetchall()
            totalrows = sumrows(table)

            if int(sumrows(table)) == int(row[0][2]):
                cell_style = 'Normal'
                cell_alignment = 'False'
                c = "All fields are populated with '" + str(row[0][0]) +"'."
                if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                    cell_style = 'Bad'
                    sqlcommand = r"""SELECT {table}.OBJECTID, {table}.{idcolumn}
                                FROM {table}
                                WHERE {table}.gs_subtype_cd =  1
                                AND ({table}.{idcolumn} IS null OR {table}.{idcolumn} = '' OR {table}.{idcolumn} LIKE '%UNK%')
                                GROUP BY {table}.OBJECTID, {table}.{idcolumn}
                                ORDER BY {table}.OBJECTID, {table}.{idcolumn}
                                """.format(idcolumn=idcolumn,table=table)
                    df = pd.read_sql_query(sqlcommand, conn)
                    sheetname = category + "-" + idcolumn[len("gs_"):]
                    createsheet(sheetname)
                    writedf(df, sheetname)
            else:
                for row in row:
                    if (row[0] == None) or (row[0] == "") or (row[0] == "UNK"):
                        cell_style = 'Bad'
                    fcol = row[0]
                    fnum = row[2]
                    fper = int(round(int(fnum)/int(totalrows)*100))
                    file_list.append({"fcol": fcol, "fnum": fnum, "fper": fper})
                cell_alignment = 'True'
                c = r""        
                c = c.join(("Populated with {fper}% ({fnum}) '{fcol}'.\n".format(fcol=fl['fcol'], fnum=fl['fnum'], fper=fl['fper']) for fl in file_list))
                if cell_style == 'Bad':
                    sqlcommand = r"""SELECT {table}.OBJECTID, {table}.{idcolumn}
                                FROM {table}
                                WHERE {table}.gs_subtype_cd =  1
                                AND ({table}.{idcolumn} IS null OR {table}.{idcolumn} ='' OR {table}.{idcolumn} LIKE '%UNK%')
                                GROUP BY {table}.OBJECTID, {table}.{idcolumn}
                                ORDER BY {table}.OBJECTID, {table}.{idcolumn}
                                """.format(idcolumn=idcolumn,table=table)
                    df = pd.read_sql_query(sqlcommand, conn)
                    sheetname = category + "-" + idcolumn[len("gs_"):]
                    createsheet(sheetname)
                    writedf(df, sheetname)
            break
        else:
            cell_style = 'Neutral'
            c = "This field needs to be added to the database."
    
    return c

In [70]:
def nullabc(idcolumn, idcolumn1, idcolumn2, idcolumn3, table):
    global cell_style 
    cell_style = 'Normal'
    global cell_alignment
    cell_alignment = 'False'
    facility_id = False
    for row in cursor.columns(table=table):
        if "gs_facility_id" in row:
            facility_id = True
    for row in cursor.columns(table=table):
        if "gs_equipment_location" in row:
            equipment_loc = True
    totalrows = sumrows(table)
    sqlcommand = r"""SELECT {table}.{idcolumn1}, {table}.{idcolumn}
                    FROM {table}
                    WHERE {table}.{idcolumn} LIKE '%a%'
                    AND ({table}.{idcolumn1} IS null OR {table}.{idcolumn1} = '' OR {table}.{idcolumn1} LIKE '%fake%' OR {table}.{idcolumn1} LIKE '%unk%')
                    ORDER BY {table}.{idcolumn};
                """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,table=table)
    
    cursor.execute(sqlcommand)
    row = cursor.fetchall()
    anum = len(row)
    aper = round(anum/int(totalrows)*100)
    c = str(aper) + "% (" + str(anum) + ") 'A', "

    sqlcommand = r"""SELECT {table}.{idcolumn2}, {table}.{idcolumn}
                FROM {table}
                WHERE {table}.{idcolumn} LIKE '%b%'
                AND ({table}.{idcolumn2} IS null OR {table}.{idcolumn2} = '' OR {table}.{idcolumn2} LIKE '%fake%' OR {table}.{idcolumn2} LIKE '%unk%')
                ORDER BY {table}.{idcolumn};
            """.format(idcolumn=idcolumn,idcolumn2=idcolumn2,table=table)

    
    cursor.execute(sqlcommand)
    row = cursor.fetchall()
    bnum = len(row)
    bper = round(bnum/int(totalrows)*100)
    c = c + str(bper) + "% (" + str(bnum) + ") 'B', "
    
    sqlcommand = r"""SELECT {table}.{idcolumn3}, {table}.{idcolumn}
                FROM {table}
                WHERE {table}.{idcolumn} LIKE '%c%'
                AND ({table}.{idcolumn3} IS null OR {table}.{idcolumn3} = '' OR {table}.{idcolumn3} LIKE '%fake%' OR {table}.{idcolumn3} LIKE '%unk%')
                ORDER BY {table}.{idcolumn};
            """.format(idcolumn=idcolumn,idcolumn3=idcolumn3,table=table)

    
    cursor.execute(sqlcommand)
    row = cursor.fetchall()
    cnum = len(row)
    cper = round(cnum/int(totalrows)*100)
    c = c + str(cper) + "% (" + str(cnum) + ") 'C' are not populated. "
    
    sqlcommand = r"""SELECT {table}.{idcolumn3}, {table}.{idcolumn}
                FROM {table}
                WHERE ({table}.{idcolumn} LIKE '%a%' OR {table}.{idcolumn} LIKE '%b%' OR {table}.{idcolumn} LIKE '%c%')
                AND ({table}.{idcolumn1} IS null OR {table}.{idcolumn1} = '' OR {table}.{idcolumn1} LIKE '%fake%' OR {table}.{idcolumn1} LIKE '%unk%')
                AND ({table}.{idcolumn2} IS null OR {table}.{idcolumn2} = '' OR {table}.{idcolumn2} LIKE '%fake%' OR {table}.{idcolumn2} LIKE '%unk%')
                AND ({table}.{idcolumn3} IS null OR {table}.{idcolumn3} = '' OR {table}.{idcolumn3} LIKE '%fake%' OR {table}.{idcolumn3} LIKE '%unk%')
                ORDER BY {table}.{idcolumn};
            """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,idcolumn2=idcolumn2,idcolumn3=idcolumn3,table=table)

    
    cursor.execute(sqlcommand)
    row = cursor.fetchall()
    tnum = len(row)
    tper = round(tnum/int(totalrows)*100)
    c = c + str(tper) + "% (" + str(tnum) + ") total."
    
    if anum + bnum + cnum == 0:
        c = "All are populated."
    else:
        cell_style = 'Bad'
        r = r""
        sqlcommand = r.join(("SELECT {table}.OBJECTID,{table}.{idcolumn},{table}.{idcolumn1},{table}.{idcolumn2},{table}.{idcolumn3}".format(idcolumn=idcolumn,idcolumn1=idcolumn1,idcolumn2=idcolumn2,idcolumn3=idcolumn3,table=table)))
        if (facility_id == True):
            sqlcommand = r.join((sqlcommand, ",{table}.gs_equipment_location".format(table=table)))
        if (facility_id == True):
            sqlcommand = r.join((sqlcommand, ",{table}.gs_facility_id".format(table=table)))
        sqlcommand = r.join((sqlcommand, """ FROM {table}
                WHERE ({table}.{idcolumn} LIKE '%a%' OR {table}.{idcolumn} LIKE '%b%' OR {table}.{idcolumn} LIKE '%c%')
                AND ({table}.{idcolumn1} IS null OR {table}.{idcolumn1} = '' OR {table}.{idcolumn1} LIKE '%fake%' OR {table}.{idcolumn1} LIKE '%unk%')
                AND ({table}.{idcolumn2} IS null OR {table}.{idcolumn2} = '' OR {table}.{idcolumn2} LIKE '%fake%' OR {table}.{idcolumn2} LIKE '%unk%')
                AND ({table}.{idcolumn3} IS null OR {table}.{idcolumn3} = '' OR {table}.{idcolumn3} LIKE '%fake%' OR {table}.{idcolumn3} LIKE '%unk%')
                ORDER BY {table}.{idcolumn};
                """.format(idcolumn=idcolumn,idcolumn1=idcolumn1,idcolumn2=idcolumn2,idcolumn3=idcolumn3,table=table)))
        df = pd.read_sql_query(sqlcommand, conn)
        sheetname = category + "-" + idcolumn1[len("gs_"):] + ",b,c"
        createsheet(sheetname)
        writedf(df, sheetname)
    return c

In [71]:
def createsheet(sheetname):
    if sheetname in wb.sheetnames:
        #wb.remove(wb[sheetname])
        #wb.create_sheet(sheetname)
        return
    else:
        wb.create_sheet(sheetname)
    return

In [72]:
def writedf(df, sheetname):
    ws = wb[sheetname]
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)
    for cell in ws[1]:
        cell.style = '40 % - Accent3'
    ws = wb["Overview"]
    return

*Writes the data to the OAMapWiseDataReview excel file.*

In [73]:
import openpyxl
import pyodbc
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

conn = pyodbc.connect("Driver={driver};Server=.\SQLEXPRESS;Database={database};Trusted_Connection=yes".format(driver = "{SQL Server}",database = "gs" + number), autocommit = True)
conn.timeout = 60
#conn.setencoding('utf-8')  # (Python 3.x syntax)
#conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
#conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
#conn.setencoding(encoding='utf-8')
cursor = conn.cursor()

cell_style = 'Normal'
cell_alignment = 'False'
cell = 3

wb = openpyxl.load_workbook(xlanalysisfile)
ws = wb["Overview"]

c = "OA Data Analysis Summary\n" + number + " " + name
ws['A1'] = c
c = "Initial pass through " + str(date.today())
ws['C1'] = c

category = "SUB"
copysubstation = 0
table = "gs_electric_station"
idcolumn = "gs_name"
c = "Substations (" + str(sumrows (table)) + ")"
ws['A2'] = c
c = uniqueid(idcolumn,table)
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_rated_voltage"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_connection_code"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)

category = "CAP"
table = "gs_capacitor_bank"
idcolumn = "gs_equipment_location"
c = "Capacitors (" + str(sumrows (table)) + ")"
ws['A8'] = c
c = uniqueid(idcolumn,table)
cell += 3
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_equipment_location"
idcolumn1 = "gs_facility_id"
c = duplicateid(idcolumn, idcolumn1, table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_unit_size_kvar"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C'+str(cell)].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_voltage_rating"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C'+str(cell)] = c
ws['C'+str(cell)].style = cell_style
ws['C13'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_status_code"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C14'] = c
ws['C14'].style = cell_style
ws['C14'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_type_code"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C15'] = c
ws['C15'].style = cell_style
ws['C15'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_connection"
c = fieldsummary(idcolumn,table)
cell += 1
ws['C16'] = c
ws['C16'].style = cell_style
ws['C16'].alignment = Alignment(wrap_text=cell_alignment)

category = "OCR"
table = "gs_overcurrent_device"
idcolumn = "gs_equipment_location"
c = "Overcurrent Devices (" + str(sumrows (table)) + ")"
ws['A21'] = c
c = uniqueid(idcolumn,table)
cell += 6
ws['C22'] = c
ws['C22'].style = cell_style
ws['C22'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_equipment_location"
idcolumn1 = "gs_phase"
c = duplicateid(idcolumn, idcolumn1, table)
ws['C23'] = c
ws['C23'].style = cell_style
ws['C23'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
ws['C24'] = c
ws['C24'].style = cell_style
ws['C24'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
idcolumn1 = "gs_device_desc_a"
idcolumn2 = "gs_device_desc_b"
idcolumn3 = "gs_device_desc_c"
c = nullabc(idcolumn, idcolumn1, idcolumn2, idcolumn3, table)
ws['C25'] = c
ws['C25'].style = cell_style
ws['C25'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_overcurrent_device_subtype"
c = fieldsummary(idcolumn,table)
ws['C26'] = c
ws['C26'].style = cell_style
ws['C26'].alignment = Alignment(wrap_text=cell_alignment)

category = "SWI"
table = "gs_switch"
idcolumn = "gs_equipment_location"
c = "Switches (" + str(sumrows (table)) + ")"
ws['A29'] = c
c = uniqueid(idcolumn,table)
ws['C30'] = c
ws['C30'].style = cell_style
ws['C30'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_equipment_location"
idcolumn1 = "gs_facility_id"
c = duplicateid(idcolumn, idcolumn1, table)
ws['C31'] = c
ws['C31'].style = cell_style
ws['C31'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
ws['C32'] = c
ws['C32'].style = cell_style
ws['C32'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_switch_status"
c = fieldsummary(idcolumn,table)
ws['C33'] = c
ws['C33'].style = cell_style
ws['C33'].alignment = Alignment(wrap_text=cell_alignment)

category = "XFMR"
table = "gs_transformer"
idcolumn = "gs_equipment_location"
c = "Transformers (" + str(sumrows (table)) + ")"
ws['A36'] = c
c = uniqueid(idcolumn,table)
ws['C37'] = c
ws['C37'].style = cell_style
ws['C37'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_equipment_location"
idcolumn1 = "gs_bank_id"
c = duplicate_xfmr(idcolumn, idcolumn1, table)
ws['C38'] = c
ws['C38'].style = cell_style
ws['C38'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
ws['C39'] = c
ws['C39'].style = cell_style
ws['C39'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_xfmr_conductor_description"
c = fieldsummary(idcolumn,table)
ws['C40'] = c
ws['C40'].style = cell_style
ws['C40'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
idcolumn1 = "gs_tran_kva_a"
idcolumn2 = "gs_tran_kva_b"
idcolumn3 = "gs_tran_kva_c"
c = nullabc(idcolumn, idcolumn1, idcolumn2, idcolumn3, table)
ws['C41'] = c
ws['C41'].style = cell_style
ws['C41'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_winding_connection"
c = fieldsummary(idcolumn,table)
ws['C42'] = c
ws['C42'].style = cell_style
ws['C42'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_rated_input_voltage"
c = fieldsummary(idcolumn,table)
ws['C43'] = c
ws['C43'].style = cell_style
ws['C43'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_rated_output_voltage"
c = fieldsummary(idcolumn,table)
ws['C44'] = c
ws['C44'].style = cell_style
ws['C44'].alignment = Alignment(wrap_text=cell_alignment)
#idcolumn = "gs_equipment_location"
#idcolumn1 = "gs_bank_id"
#c = duplicateid(idcolumn, idcolumn1, table) + " They should have gs_bank_ids added if they are banked."
#ws['C45'] = c
#ws['C45'].style = cell_style
#ws['C45'].alignment = Alignment(wrap_text=cell_alignment)

category = "VREG"
table = "gs_voltage_regulator"
idcolumn = "gs_equipment_location"
c = "Voltage Regulators (" + str(sumrows (table)) + ")"
ws['A53'] = c
c = uniqueid(idcolumn,table)
ws['C54'] = c
ws['C54'].style = cell_style
ws['C54'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_equipment_location"
idcolumn1 = "gs_facility_id"
c = duplicateid(idcolumn, idcolumn1, table)
ws['C55'] = c
ws['C55'].style = cell_style
ws['C55'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
ws['C56'] = c
ws['C56'].style = cell_style
ws['C56'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
idcolumn1 = "gs_regulator_a"
idcolumn2 = "gs_regulator_b"
idcolumn3 = "gs_regulator_c"
c = nullabc(idcolumn, idcolumn1, idcolumn2, idcolumn3, table)
ws['C57'] = c
ws['C57'].style = cell_style
ws['C57'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_winding_connection"
c = fieldsummary(idcolumn,table)
ws['C58'] = c
ws['C58'].style = cell_style
ws['C58'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_nominal_voltage"
c = fieldsummary(idcolumn,table)
ws['C59'] = c
ws['C59'].style = cell_style
ws['C59'].alignment = Alignment(wrap_text=cell_alignment)

category = "SPAN"
table = "gs_span"
c = "Conductors (" + str(sumrows (table)) + ")"
ws['A74'] = c
idcolumn = "gs_phase"
c = fieldsummary(idcolumn,table)
ws['C75'] = c
ws['C75'].style = cell_style
ws['C75'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_phase"
idcolumn1 = "gs_conductor_a"
idcolumn2 = "gs_conductor_b"
idcolumn3 = "gs_conductor_c"
c = nullabc(idcolumn, idcolumn1, idcolumn2, idcolumn3, table)
ws['C76'] = c
ws['C76'].style = cell_style
ws['C76'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_conductor_n"
c = neutral(idcolumn,table)
ws['C77'] = c
ws['C77'].style = cell_style
ws['C77'].alignment = Alignment(wrap_text=cell_alignment)
idcolumn = "gs_subtype_cd"
c = fieldsummary(idcolumn,table)
ws['C78'] = c
ws['C78'].style = cell_style
ws['C78'].alignment = Alignment(wrap_text=True)
idcolumn = "gs_construction_desc"
c = missingfield(idcolumn,table)
ws['C79'] = c + " Can use RUS standards here."
ws['C79'].style = cell_style
ws['C79'].alignment = Alignment(wrap_text=cell_alignment)
ws.column_dimensions['C'].auto_size = True
print("Done.")
conn.close()
wb.save(xlanalysisfile)
wb.close()

Done.
